In [1]:
import mamba_ssm
import torch
import matplotlib.pyplot as plt
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    return total_params

/home/ubuntu/Test/miniforge-pypy3/envs/mamba/lib/python3.11/site-packages/mamba_ssm/ops/selective_scan_interface.py:163: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/home/ubuntu/Test/miniforge-pypy3/envs/mamba/lib/python3.11/site-packages/mamba_ssm/ops/selective_scan_interface.py:239: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/home/ubuntu/Test/miniforge-pypy3/envs/mamba/lib/python3.11/site-packages/mamba_ssm/ops/triton/layer_norm.py:985: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/home/ubuntu/Test/miniforge-pypy3/envs/mamba/lib/python3.11/site-packages/mamba_ssm/ops/triton/layer_norm.py:1044: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Ple

In [2]:
#from mamba import Mamba2
!ls Test/mamba
from Test.mamba.mamba_ssm.modules.mamba2 import Mamba2

AUTHORS    assets      evals		   pyproject.toml  tests
LICENSE    benchmarks  mamba_ssm	   rocm_patch
README.md  csrc        mamba_ssm.egg-info  setup.py


In [3]:
layer = Mamba2(256)

In [4]:
layer.to('cuda')

Mamba2(
  (in_proj): Linear(in_features=256, out_features=1288, bias=False)
  (conv1d): Conv1d(768, 768, kernel_size=(4,), stride=(1,), padding=(3,), groups=768)
  (act): SiLU()
  (norm): RMSNorm()
  (out_proj): Linear(in_features=512, out_features=256, bias=False)
)

In [5]:
layer(torch.rand([2,2048,256],device='cuda'))

Forward
Using mem_eff_path combined conv1d/scan


tensor([[[ 1.3419,  0.1934,  0.9124,  ...,  0.0342, -0.1520,  0.5218],
         [ 0.5760,  0.9400,  1.0234,  ..., -0.3908,  0.3416,  0.2747],
         [ 1.2030,  0.0720,  0.4944,  ..., -0.2286,  0.0182,  0.2326],
         ...,
         [ 0.6874,  0.1012,  0.2480,  ...,  0.6149,  0.1116,  0.2490],
         [ 1.3338,  0.3831,  0.0687,  ..., -0.5297, -0.6489,  0.9802],
         [ 0.2793,  0.1719,  0.3525,  ..., -0.0395,  0.1204,  0.4159]],

        [[ 1.0883,  0.4892,  0.3834,  ..., -0.1507, -0.2160,  0.4070],
         [ 0.9860,  0.2323,  0.6860,  ..., -0.0164, -0.0222,  0.2888],
         [ 0.9173,  1.0618,  0.4473,  ...,  0.3588, -0.0849,  0.2178],
         ...,
         [ 0.6079, -0.1363,  0.4662,  ...,  0.2893,  0.1701,  0.4217],
         [ 0.8150,  0.2136,  0.6767,  ...,  0.2821, -0.6732,  0.8068],
         [ 1.0187,  0.4325,  0.3621,  ..., -0.1710, -0.6066,  0.7537]]],
       device='cuda:0', grad_fn=<MambaSplitConv1dScanCombinedFnBackward>)

In [ ]:
import math
from collections import defaultdict
pow = range(6,12)
print([2**p for p in pow])
pdict = defaultdict(list)
for apow in pow:
    for n, p in Mamba2(2**apow).named_parameters():
        pdict[n].append(math.log(p.numel()))
for k,v in pdict.items():
    plt.figure()
    plt.plot(pow,v)
    plt.title(k)

In [6]:
import torch.distributed as dist
dist.init_process_group(8)

ValueError: Backend constructor parameter must be string-ish